In [1]:
%pip install torchsummary

  Using cached torchsummary-1.5.1-py3-none-any.whl.metadata (296 bytes)
Using cached torchsummary-1.5.1-py3-none-any.whl (2.8 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from torch.utils.data import (DataLoader,)  # Gives easier dataset managment and creates mini batches
#import torchvision  # torch package for vision related things
#import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
#import torchvision.transforms as transforms
import pandas as pd
import random
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import RandomizedSearchCV
from tqdm.auto import tqdm
from transformers import DistilBertTokenizer
from torchsummary import summary

# Data Loading
Grabbing the dataset with the like count included

In [3]:
# Split the downloaded Youtube01-Psy.csv file into train, validation and test sets
data = pd.read_csv('SpamDataset/TSwift_ShakeItOff_Spam.csv')
# data = pd.read_csv('TSwift_ShakeItOff_Spam.csv')
data = data.sample(frac=1).reset_index(drop=True)  # Shuffle the data
train_data = data[:int(0.7*len(data))]
val_data = data[int(0.7*len(data)):int(0.85*len(data))]
test_data = data[int(0.85*len(data)):]

# Tokenize the data using distillbert
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Turn pd dataframe into a tokenized PyTorch dataset that DataLoader can use via the SpamDataset class
class SpamDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        comment_tokens = row['CONTENT']
        like_count = row['LIKE_COUNT']
        summary = row['SUMMARY']
        label = row['CLASS']
        return comment_tokens, like_count, summary, label
    
    def collate_fn(self, batch):
        comments = [row[0] for row in batch]
        like_counts = [row[1] for row in batch]
        summary = [row[2] for row in batch]
        labels = [row[3] for row in batch]
        comment_tokens = tokenizer.batch_encode_plus(comments, truncation=True, padding='max_length', max_length=70, return_tensors='pt')
        summary_tokens = tokenizer.batch_encode_plus(summary, truncation=True, padding='max_length', max_length=70, return_tensors='pt')
        return comment_tokens, torch.tensor(like_counts), summary_tokens, torch.tensor(labels)
    
train_dataset = SpamDataset(train_data)
val_dataset = SpamDataset(val_data)
test_dataset = SpamDataset(test_data)


# Load Data and collate it
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=train_dataset.collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=val_dataset.collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=test_dataset.collate_fn)

In [4]:
# Print a random sample from the train_loader to confirm correct loading and tokenization
sample = next(iter(train_loader))
print(tokenizer.decode(sample[0]['input_ids'][0]))
print(tokenizer.decode(sample[2]['input_ids'][0]))

[CLS] the day this video was released was the day i quit drugs. i ’ ll never forget it. this song and “ clover cage - learning to love myself ” are the two songs that helped get rid of my depression. i just want to send love to everyone and i want everyone to know things will get better. we are in this together [SEP]
[CLS] taylor swift's " shake it off " music video is a colorful, upbeat anthem celebrating self - confidence and positivity. with catchy lyrics and energetic choreography, swift embraces different dance styles and encourages listeners to shrug off negativity. the video's vibrant visuals and playful tone make it a feel - good pop sensation. [SEP]


# Model
- Model 1 - include Like count
- Model 2 - include LLM evaluation of the video 

In [5]:
## The network includes an Embedding layer, an Attention layer, an LSTM layer and a Linear layer
class CombinedLSTM(nn.Module):
    def __init__(self, embedding_dict, embedding_size, hidden_size, num_layers, num_classes, dropout):
        super(CombinedLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(embedding_dict, embedding_size)
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.attention = nn.Linear(hidden_size, 1)


        self.embedding_summary = nn.Embedding(embedding_dict, embedding_size)
        self.dropout_summary = nn.Dropout(dropout)
        self.lstm_summary = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.attention_summary = nn.Linear(hidden_size, 1)


        self.fc = nn.Linear(2 * hidden_size + 1, num_classes) # +1 for like count
        
    def forward(self, comment_sequence, like_count, summary_sequence):
        # Evaluate the comment sequence
        # comment_sequence: (batch_size, max_seq_length)
        comment_embedding = self.dropout(self.embedding(comment_sequence))
        # embedding shape: (batch_size, max_seq_length, embedding_size)
        output_comments, (h_n, c_n) = self.lstm(comment_embedding)
        # output shape: (batch_size, max_seq_length, hidden_size)
        # h_n shape: (num_layers, batch_size, hidden_size)
        attention_weights = F.softmax(self.attention(output_comments), dim=1)
        # attention_weights shape: (batch_size, max_seq_length, 1)
        output_comments = torch.sum(attention_weights * output_comments, dim=1)

        # Evaluate the summary sequence
        # summary_sequence: (batch_size, max_seq_length)
        summary_embedding = self.dropout_summary(self.embedding_summary(summary_sequence))
        # embedding shape: (batch_size, max_seq_length, embedding_size)
        output_summary, (h_n, c_n) = self.lstm_summary(summary_embedding)
        # output shape: (batch_size, max_seq_length, hidden_size)
        # h_n shape: (num_layers, batch_size, hidden_size)
        attention_weights = F.softmax(self.attention_summary(output_summary), dim=1)
        # attention_weights shape: (batch_size, max_seq_length, 1)
        output_summary = torch.sum(attention_weights * output_summary, dim=1)

        # Combine the comment and summary feature outputs with the like count
        # out shape: (batch_size, hidden_size)
        out = torch.cat((output_comments, output_summary, like_count.unsqueeze(1)), 1)
        # out shape: (batch_size, hidden_size + 1)
        out = self.fc(out)
        # out shape: (batch_size, num_classes)
        return out

# Training

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
# Set the hyperparameters for all the models     # TODO: analyze these by model
embedding_dict = tokenizer.vocab_size
embedding_size = 128
hidden_size = 256
num_layers = 4
num_classes = 2
dropout = 0.4
learning_rate = 0.0007
num_epochs = 15

# Initialize the model, loss function and optimizer
model = CombinedLSTM(embedding_dict, embedding_size, hidden_size, num_layers, num_classes, dropout).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(train_loader):
        comment_tokens, like_counts, summary_tokens, labels = batch
        comment_sequence = comment_tokens['input_ids'].to(device)
        like_counts = like_counts.to(device)
        summary_sequence = summary_tokens['input_ids'].to(device)
        labels = labels.to(device)
        outputs = model(comment_sequence, like_counts, summary_sequence)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1/15, Loss: 0.5169246792793274


  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 2/15, Loss: 0.4684121906757355


  0%|          | 0/6 [00:00<?, ?it/s]

...

Epoch 13/15, Loss: 0.24954117834568024


  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 14/15, Loss: 0.06824345141649246


  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 15/15, Loss: 0.30537575483322144


# Validation

In [7]:
# Validation
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    predictions = []
    targets = []
    model.eval()

    with torch.no_grad():
        for comments, like_counts, summary, target in loader:
            comments = {key: value.to(device) for key, value in comments.items()}
            like_counts = like_counts.to(device)
            summary = {key: value.to(device) for key, value in summary.items()}
            target = target.to(device)

            scores = model(comments['input_ids'], like_counts, summary['input_ids'])
            _, predicted = scores.max(1)
            # Store decoded original comments, model predictions and actual labels 
            num_correct += (predicted == target).sum()
            num_samples += predicted.size(0)

            predictions.extend(predicted.tolist())
            targets.extend(target.tolist())

    # model.train()
    accuracy = num_correct/num_samples
    return accuracy, predictions, targets

def results(model):
    train_acc, train_preds, train_targets = check_accuracy(train_loader, model)
    val_acc, val_preds, val_targets = check_accuracy(val_loader, model)
    test_acc, test_preds, test_targets = check_accuracy(test_loader, model)

    print(f'Train accuracy: {train_acc}')
    print(f'Validation accuracy: {val_acc}')
    print(f'Test accuracy: {test_acc}')

    train_cm = confusion_matrix(train_targets, train_preds)
    val_cm = confusion_matrix(val_targets, val_preds)
    test_cm = confusion_matrix(test_targets, test_preds)

    print(f'Train Confusion Matrix:\n{train_cm}')
    print(f'Validation Confusion Matrix:\n{val_cm}')
    print(f'Test Confusion Matrix:\n{test_cm}')

    # Classification report
    train_report = classification_report(train_targets, train_preds)
    val_report = classification_report(val_targets, val_preds)
    test_report = classification_report(test_targets, test_preds)

    print(f'Train Classification Report:\n{train_report}')
    print(f'Validation Classification Report:\n{val_report}')
    print(f'Test Classification Report:\n{test_report}')

In [8]:
results(model)

Train accuracy: 0.976401150226593
Validation accuracy: 0.8767123222351074
Test accuracy: 0.8219178318977356
Train Confusion Matrix:
[[222   2]
 [  6 109]]
Validation Confusion Matrix:
[[42  6]
 [ 3 22]]
Test Confusion Matrix:
[[41  9]
 [ 4 19]]
Train Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       224
           1       0.98      0.95      0.96       115

    accuracy                           0.98       339
   macro avg       0.98      0.97      0.97       339
weighted avg       0.98      0.98      0.98       339

Validation Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.88      0.90        48
           1       0.79      0.88      0.83        25

    accuracy                           0.88        73
   macro avg       0.86      0.88      0.87        73
weighted avg       0.88      0.88      0.88        73

Test Classification Report:
          

In [11]:
from torchviz import make_dot
embedding_dict = tokenizer.vocab_size
embedding_size = 128
hidden_size = 256
num_layers = 1
num_classes = 2
dropout = 0.4
learning_rate = 0.0007
num_epochs = 15

# Initialize the model, loss function and optimizer
model = CombinedLSTM(embedding_dict, embedding_size, hidden_size, num_layers, num_classes, dropout).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

batch = next(iter(train_loader))
scores = model(batch[0]['input_ids'], batch[1], batch[2]['input_ids'])

print(model)
make_dot(scores, params=dict(list(model.named_parameters()))).render("model_images/feature_combination_lstm", format="png")

TypeError: 'int' object is not iterable